# Who's winning it? Forecasting sports tournaments

*S3, Pozega, Croatia. July 19-27, 2017*

The goal of this project is to build a probabilistic forecast of the EURO 2016 football tournament using the historical data about international football matches. The data are used to estimate team strength and to build a goal prediction model. The forecast probabilities are estimated with Monte Carlo simulation.

## Building the data-driven components of the system

We have got the scores for all international football matches since 1870s. We first use these data to estimate team strength using the Elo ranking method. Then, we use the Elo ratings to build a goal prediction model.

### Loading the data

In [ ]:
import pandas as pd

In [ ]:
scores = pd.read_csv('international+euro16.csv',
                     names=['type_long', 'type', 'date', 'team1', 'team2', 'goals1', 'goals2'],
                     sep=',', header=None,
                     parse_dates=[2], index_col=[2])

In [ ]:
print(scores.shape)
print(scores.dtypes)
scores.head()

### Building the Elo ranking

The Elo rating of a team represents its strength. The probability that Team 1 with rating $R_1$ beats Team 2 with rating $R_2$ at home is calculated as follows: $$P(\text{Team 1 beats Team 2}) = \frac{1}{1+{10}^{-(R_1+H-R_2)/400}}$$

The rating of a team is updated after each match based on the strength of the opponent, the outcome, and a number of parameters: $$\Delta R_1=(O_\text{actual}-O_\text{expected}) \times K \times I \times N \times G$$

In [ ]:
Rinitial=1500

def expected_outcome(r, ro):
    return  1/(1+10**((ro-r)/400))

def probabilities(exp_outcome):
    p_draw = exp_outcome * (1 - exp_outcome) * 4/3
    p_win = exp_outcome - 0.5 * p_draw
    p_loss = 1 - p_draw - p_win
    return { 1: p_win, 0.5: p_draw, 0: p_loss }

def outcome(g,go):
    if g > go:
        outcome = 1
    elif g < go:
        outcome = 0
    else:
        outcome = 0.5
    
    return (outcome)

def updateElo(r,ro,g,go,game_type,K,I):
    actual_outcome = outcome(g,go)
    expected_outcome_ = expected_outcome(r,ro)
    
    if g >= 5:
        N = 1
    elif g == 4:
        N = 0.85
    elif g == 3:
        N = 0.60
    elif g == 2:
        N = 0.50
    else:
        N = 0.40
    
    goal_difference = abs(g-go)
    if goal_difference >= 5:
        G=1.5
    elif goal_difference == 4:
        G=1.35
    elif goal_difference == 3:
        G=1.20
    elif goal_difference == 2:
        G=1.05
    else:
        G=1
    
    return (actual_outcome - expected_outcome_)*K*I[game_type]*N*G

def newElo(r1,r2,g1,g2,game_type,K,I,H):
    if game_type == 'Friendly' or 'Qualifier' in game_type:
        r1_ = r1 + H
    else:
        r1_ = r1
        
    return r1 + updateElo(r1_,r2,g1,g2,game_type,K,I), r2 + updateElo(r2,r1_,g2,g1,game_type,K,I)

def eloRatings(scores,K,I,H,return_elo_columns=False):
    '''Building Elo ratings using `scores` as the training data and `K` and `H` as parameters'''
    elos = {}
    elos1 = []
    elos2 = []

    for index, row in scores.iterrows():
        t1 = row['team1']
        t2 = row['team2']
        r1 = elos.get(t1, Rinitial)
        r2 = elos.get(t2, Rinitial)
        
        if return_elo_columns:
            elos1.append(r1)
            elos2.append(r2)
        
        r1_new, r2_new = newElo(r1, r2, row['goals1'], row['goals2'],row['type_long'],K,I,H)
        elos[t1] = r1_new
        elos[t2] = r2_new

    return elos, elos1, elos2

def top_elo_teams(ratings,k=10):
    '''Return top-`k` teams according to Elo ratings in `ratings`'''
    by_rating = sorted(ratings.keys(), key=lambda country: ratings[country], reverse=True)
    return [(country, ratings[country]) for country in by_rating[0:k]]

In [ ]:
equal_game_type_weights = {
    'Africa':1 ,
    'AfricaQualifier':1 ,
    'America':1 ,
    'AmericaQualifier':1 ,
    'Asia':1 ,
    'AsiaQualifier':1 ,
    'Confederations':1 ,
    'ConfederationsQualifier':1 ,
    'Europe':1 ,
    'EuropeQualifier':1 ,
    'Friendly':1 ,
    'Oceania':1 ,
    'OceaniaQualifier':1 ,
    'SouthAmerica':1 ,
    'SouthAmericaQualifier':1 ,
    'World':1 , 
    'WorldQualifier':1
}

In [ ]:
elo1, elo2 = [1767, 1686]
print(expected_outcome(elo1, elo2))
print(newElo(elo1, elo2, 2, 1, 'Friendly', K=30, I=equal_game_type_weights, H=0))

#### Tuning the parameters

We split the entire data into training data and validation data. We built the ratings using the training set and pick the parameter values that yield the lowest prediction error on the validation set as measured by *log loss*.

In [ ]:
train_end   = '2013-12-31'

validation_start = '2014-01-01'
validation_end   = '2016-06-08'
validation_set   = scores[validation_start:validation_end]
validation_size  = validation_set.shape[0]

test_start = '2016-06-10'
test_set   = scores[test_start:]
test_size  = test_set.shape[0]

validation_size, test_size

In [ ]:
from math import log10

def logLoss(p, eps=10**(-15)):
    p_nonzero = max(min(p, 1-eps), eps)
    return log10(p_nonzero)

In [ ]:
random_guess_loss = -logLoss(1/3)
random_guess_loss

In [ ]:
def elo_log_loss(ratings_, matches, K, I, H):
    num_matches = matches.shape[0]
    
    ratings=ratings_.copy()
    log_loss = 0
    for index, row in matches.iterrows():
        t1 = row['team1']
        t2 = row['team2']
        r1 = ratings.get(t1, Rinitial)
        r2 = ratings.get(t2, Rinitial)
        
        g1 = row['goals1']
        g2 = row['goals2']
        
        actual_outcome=outcome(g1,g2)
        forecast=probabilities(expected_outcome(r1,r2))[actual_outcome]
        log_loss = log_loss + logLoss(forecast)
        
        r1_new, r2_new = newElo(r1, r2, row['goals1'], row['goals2'], row['type_long'], K=K, I=I, H=H)
        ratings[t1] = r1_new
        ratings[t2] = r2_new
        
    return -log_loss / num_matches

In [ ]:
different_type_weights = [
    equal_game_type_weights,
    {'Africa': 1.25,
     'AfricaQualifier': 1,
     'America': 1.25,
     'AmericaQualifier': 1,
     'Asia': 1.25,
     'AsiaQualifier': 1,
     'Confederations': 2,
     'ConfederationsQualifier': 2,
     'Europe': 3,
     'EuropeQualifier': 2.5,
     'Friendly': 1,
     'Oceania': 1.5,
     'OceaniaQualifier': 1.25,
     'SouthAmerica': 2.75,
     'SouthAmericaQualifier': 2.4,
     'World': 3.5,
     'WorldQualifier': 2.75,
     },
    {'Africa': 0.6,
     'AfricaQualifier': 0.35,
     'America': 0.6,
     'AmericaQualifier': 0.35,
     'Asia': 0.6,
     'AsiaQualifier': 0.35,
     'Confederations': 1,
     'ConfederationsQualifier': 1,
     'Europe': 1.25,
     'EuropeQualifier': 1,
     'Friendly': 0.25,
     'Oceania': 0.5,
     'OceaniaQualifier': 0.25,
     'SouthAmerica': 1.15,
     'SouthAmericaQualifier': 0.85,
     'World': 1.5,
     'WorldQualifier': 1,
     },
    {'Africa': 3,
     'AfricaQualifier': 2,
     'America': 2.5,
     'AmericaQualifier': 1.75,
     'Asia': 2.5,
     'AsiaQualifier': 1.75,
     'Confederations': 4,
     'ConfederationsQualifier': 4,
     'Europe': 4.5,
     'EuropeQualifier': 3.5,
     'Friendly': 1.5,
     'Oceania': 2.5,
     'OceaniaQualifier': 1.75,
     'SouthAmerica': 4.75,
     'SouthAmericaQualifier': 3.85,
     'World': 5,
     'WorldQualifier': 4,
     },
    {'Africa': 1.25,
     'AfricaQualifier': 1,
     'America': 1.25,
     'AmericaQualifier': 1,
     'Asia': 1.25,
     'AsiaQualifier': 1,
     'Confederations': 2,
     'ConfederationsQualifier': 2,
     'Europe': 2.5,
     'EuropeQualifier': 2,
     'Friendly': 1,
     'Oceania': 1.5,
     'OceaniaQualifier': 1.25,
     'SouthAmerica': 2.5,
     'SouthAmericaQualifier': 2,
     'World': 3,
     'WorldQualifier': 2.5,
     }
]

In [ ]:
# from tqdm import tqdm_notebook as tqdm
def tqdm(x, **kwargs): return x

In [ ]:
# parameter_tuning = 'Fixed values'
# parameter_tuning = 'Full search'
parameter_tuning = 'Tune individual parameters'

if parameter_tuning == 'Fixed values':
    best_parameters = [50, 1973, 100, equal_game_type_weights]
    print(best_parameters[:3])
    print(best_parameters[3])
    
    best_K, best_year_start, best_H, best_I = best_parameters
    best_train_start = str(best_year_start) + '-1-1'
    
    best_train_set = scores[best_train_start:train_end]
    best_elos      = eloRatings(best_train_set,K=best_K,I=best_I,H=best_H)[0]
    best_elo_loss  = elo_log_loss(best_elos,validation_set,K=best_K,I=best_I,H=best_H)
    print('\nLog loss on validation set: ELO={0:.4f} RANDOM={1:.4f}'.format(best_elo_loss, random_guess_loss))
elif parameter_tuning == 'Full search':
    best_log_loss = 2
    best_parameters = []

    for K in tqdm([1, 5, 10, 16, 25, 50, 56], desc='K', leave=False):
        for year_start in tqdm(range(1973,2007,5), desc='Start year', leave=False):
            for H in tqdm([0,50,60,100], desc='H', leave=False):
                for I in tqdm(different_type_weights, desc='I', leave=False):
                    train_start = str(year_start) + '-1-1'
                    train_set   = scores[train_start:train_end]

                    elos_initial = eloRatings(train_set,K,I,H)[0]
                    elo_loss     = elo_log_loss(elos_initial,validation_set,K,I,H)

                    if elo_loss < best_log_loss:
                        best_log_loss = elo_loss
                        best_parameters = [K, year_start, H, I]

    print(best_log_loss, random_guess_loss)
    for p in best_parameters:
        print(p)

    best_K, best_year_start, best_H, best_I = best_parameters
    best_train_start = str(best_year_start) + '-1-1'
elif parameter_tuning == 'Tune individual parameters':
    best_log_loss = 2
    best_train_start, best_H, best_I = '1973-1-1', 0, equal_game_type_weights
    
    for K in tqdm([1, 5, 10, 16, 25, 50, 56], desc='K', leave=False):
        train_set   = scores[best_train_start:train_end]

        elos_initial = eloRatings(train_set,K,best_I,best_H)[0]
        elo_loss     = elo_log_loss(elos_initial,validation_set,K,best_I,best_H)

        if elo_loss < best_log_loss:
            best_log_loss = elo_loss
            best_K = K
    print('            ERROR  VALUE')
    print('Best K:     {1:.4f} {0:d}'.format(best_K, best_log_loss))
    
    for year_start in tqdm(range(1974,2007,5), desc='Start year', leave=False):
        train_start = str(year_start) + '-1-1'
        train_set   = scores[train_start:train_end]

        elos_initial = eloRatings(train_set,best_K,best_I,best_H)[0]
        elo_loss     = elo_log_loss(elos_initial,validation_set,best_K,best_I,best_H)

        if elo_loss < best_log_loss:
            best_log_loss = elo_loss
            best_train_start = str(year_start) + '-1-1'
    print('Best start: {1:.4f} {0:s}'.format(best_train_start, best_log_loss))
    
    for H in tqdm([0,50,60,100], desc='H', leave=False):
        train_set   = scores[best_train_start:train_end]

        elos_initial = eloRatings(train_set,best_K,best_I,H)[0]
        elo_loss     = elo_log_loss(elos_initial,validation_set,K,best_I,H)

        if elo_loss < best_log_loss:
            best_log_loss = elo_loss
            best_H = H
    print('Best H:     {1:.4f} {0:d}'.format(best_H, best_log_loss))
        
    for I in tqdm(different_type_weights, desc='I', leave=False):
        train_set   = scores[best_train_start:train_end]

        elos_initial = eloRatings(train_set,best_K,I,best_H)[0]
        elo_loss     = elo_log_loss(elos_initial,validation_set,best_K,I,best_H)

        if elo_loss < best_log_loss:
            best_log_loss = elo_loss
            best_I=I
    print('Best I:     {1:.4f} {0:d}'.format(different_type_weights.index(best_I), best_log_loss))
    # print(best_I)

The error of the forecasting system based on the estimated rating is `0.40`, lower than the error of random guessing of `0.48`. Note that validation shows that accounting for match importance does **not** improve the performance of the system.

In [ ]:
full_train_set = scores[best_train_start:validation_end].copy()
elos, elo1, elo2 = eloRatings(full_train_set, K=best_K, H=best_H, I=best_I, return_elo_columns=True)
print('Size of the full training set (train+validation): {0:d} matches'.format(full_train_set.shape[0]))
print('Size of the test set (EURO 2016): {0:d} matches\n'.format(test_set.shape[0]))

full_train_set['elo1'] = elo1
full_train_set['elo2'] = elo2

test_loss = elo_log_loss(elos, test_set, K=best_K, H=best_H, I=best_I)
print('Log loss on the test set:\n   ELO={0:.4f}\nRANDOM={1:.4f}'.format(test_loss, random_guess_loss))

In [ ]:
for team, elo in top_elo_teams(elos, k=25):
    print('{0:25s} = {1:.1f}'.format(team, elo))

### Building the goal prediction model

We use the training data and the Elo ratings to build the goal prediction model. Using Poisson regression, we estimate the effect of the strength of a given team and its opponent on the number of goals scored. We simulate goals scored with the predicted number of goals as $\lambda$, the parameter of the Poisson distribution.

Note that each match generates two examples for Poisson regression.

In [ ]:
scores_reversed = full_train_set[['elo2','elo1','goals2']]
scores_reversed.columns = ['elo1', 'elo2', 'goals1']

elo_goals = full_train_set[['elo1','elo2','goals1']].append(scores_reversed)
elo_goals.columns = ['elo_team', 'elo_opponent', 'goals_team']

elo_goals.head()

In [ ]:
from statsmodels.genmod.generalized_estimating_equations import GEE
from statsmodels.genmod.cov_struct import Independence
from statsmodels.genmod.families import Poisson

In [ ]:
poisson_regression = GEE.from_formula("goals_team ~ elo_team + elo_opponent", data=elo_goals, 
                                      groups=list(range(0, full_train_set.shape[0])) * 2, # Two examples from a match form 
                                                                                          # a group in Poisson regression
                                                                                          # (not used in this project)
                                      cov_struct=Independence(), family=Poisson())
goals_predictor = poisson_regression.fit()
print(goals_predictor.summary())

In [ ]:
import numpy as np

In [ ]:
def simulate_goals(elo_team, elo_opponent):
    goals_lambda = goals_predictor.predict(pd.DataFrame([{'elo_team': elo_team, 'elo_opponent': elo_opponent}]))
    return np.random.poisson(goals_lambda)

def simulate_match(elo1, elo2):
    return simulate_goals(elo1, elo2), simulate_goals(elo2, elo1)

Even though we do not enforce that win probabilities resulting from Poisson simulations are consistent with the probabilities directly predicted by the Elo ratings, they are remarkably close.

In [ ]:
elo1, elo2 = elos['Bulgaria'], elos['Greece']
elo_prediction = probabilities(expected_outcome(elo1, elo2))

def outcome_tuple(pair_of_goals):
    return outcome(pair_of_goals[0], pair_of_goals[1])

N=1000
poisson_outcomes, poisson_counts = np.unique(np.array([outcome_tuple(simulate_match(elo1, elo2)) for i in range(0, N)]), 
                                             return_counts=True)
poisson_frequencies = poisson_counts / N
poisson_prediction = {}
for o in range(0, np.size(poisson_outcomes)):
    poisson_prediction[poisson_outcomes[o]] = poisson_frequencies[o]

for o,p in elo_prediction.items():
    print('{0:.1f} ELO={1:.2f} POISSON={2:.2f}'.format(o, p, poisson_prediction[o]))

## Building the simulator

The simulator comprises two major components. The first component simulates the group stage, including the tiebreaking rules. The second component simulates the knockout stage

### Simulating the group stage
See Wikipedia ([1](https://en.wikipedia.org/wiki/UEFA_Euro_2016#Tiebreakers), 
[2](https://en.wikipedia.org/wiki/UEFA_Euro_2016#Ranking_of_third-placed_teams),
[3](https://en.wikipedia.org/wiki/UEFA_Euro_2016_knockout_phase#Format)) 
for the tournament scheme and the description of tie-breaking rules.

In [ ]:
group_country = [
    ['France','Romania','Albania','Switzerland'],
    ['England','Russia','Wales','Slovakia'],
    ['Germany','Ukraine','Poland','Northern Ireland'],
    ['Spain','Czechia','Turkey','Croatia'],
    ['Belgium','Italy','Ireland','Sweden'],
    ['Portugal','Iceland','Austria','Hungary']
]

In [ ]:
from functools import cmp_to_key

In [ ]:
def _simulate_goals(name1,name2):
    return simulate_match(elos[name1],elos[name2])
    
def points(goals_team, goals_opponnent):
    if (goals_team > goals_opponnent):
        return 3
    elif goals_team == goals_opponnent:
        return 1
    else:
        return int(0)

def diff(g1,g2):
    return g1-g2

def simulate_score(i):
    _scores = []
    for t in range(0, 4):
        for opp in range(t + 1, 4):
            goals_t, goals_opp = _simulate_goals(group_country[i][t],group_country[i][opp])
            _scores.append([t, opp, goals_t, goals_opp])
    return _scores


def calculate_points(scores, t):
    p1=sum([points(scores[2], scores[3]) for scores in scores if scores[0] == t])
    p2=sum([points(scores[3], scores[2]) for scores in scores if scores[1] == t])
    return p1+p2

def group_points(t):
    _points = []
    for i in range (0,4):
        _points.append(calculate_points(t,i))
    return _points

def com_po(i):
    return group_points(simulate_score(i))
        
def duplicates(_points, item):
    return [i for i, x in enumerate(_points) if x == item] 

def finddpu(_points):
    for i in range (0,10):
        dup = duplicates(_points,i)
        if len(dup) == 3:
            return dup

def get_goals(score, t):
    return sum([diff(score[2],score[3]) for score in score if score[0] == t]) +\
        sum ([diff(score[3],score[2]) for score in score if score[1] == t])
    
def group_diff(t):
    _goal_diff = []
    for i in range (0,4):
        _goal_diff.append (get_goals(t,i))
    return _goal_diff

def com_dif (i):
    return group_diff(simulate_score(i))

def simulate_groups():
    global_scores = []
    global_diff = []
    _3points = []
    grouporder = []
    for i in range(0,6):
        _scores = simulate_score(i)
        _points = com_po(i)
        _goal_diff = com_dif(i)
        global_diff.append (_goal_diff[2])
        _3points.append (_points[2])
        global_scores.append (_scores)
        def simulate_group(points):
                def sorting_rules1():
                    def compare_teams1(t1,t2):
                        if points[t1] != points[t2]: 
                            return (int(points[t1])-int(points[t2]))
                        else:
                            return 0
                    return  sorted(range(0,4),key=cmp_to_key(compare_teams1), reverse=True)  
                def sorting_rules3(order, scores, points):
                    def compare_teams3(t1,t2):
                        t1goals = 0
                        t2goals = 0
                        for sc in scores: 
                            i1 = sc.index(t1) if t1 in sc[0:2] else None
                            i2 = sc.index(t2) if t2 in sc[0:2] else None
                            if i1 is not None and i2 is not None:
                                t1goals = sc[i1+2]
                                t2goals = sc[i2+2]
                            else:
                                pass
                        return t1goals-t2goals
                    return sorted(range(0,4),key=cmp_to_key(compare_teams3), reverse=True)  
                return sorting_rules3(sorting_rules1(),_scores,_points)
        grouporder.append (simulate_group(_points))
    def find3_team():
        teams3 = []
        for i in range (0,6):
            teams3.append(grouporder[i][2])
        return teams3
    def find_qual12():     
        country_key = []
        qual_teams = []
        for i in range (0,6):
            for n in range(0,2):
                qual_teams.append ([group_country[i][grouporder[i][n]],str(i)+str(n)])
        return qual_teams
    def find_qual32():
        _3team = []
        for i in range (0,4):
            _3team.append ([group_country[i][grouporder[i][2]],str(i)+str(2)])
        return _3team
    def compare_teams4(t1,t2):
        if _3points[t1] != _3points[t2]: 
            return (int(_3points[t1])-int(_3points[t2]))
        else:
            return 0
    def sorting_rules4():
        return sorted(range(0,6),key=cmp_to_key(compare_teams4), reverse=True)
    return grouporder,sorted(range(0,6),key=cmp_to_key(compare_teams4), reverse=True),find_qual32(),find_qual12()

def simulate_group_stage():
    _, _, third_placed, first_second_placed = simulate_groups()
    
    qualified = {}
    for team, position_code in third_placed:
        qualified[position_code] = team
    for team, position_code in first_second_placed:
        qualified[position_code] = team
    
    return qualified

Simulate the group stage once:

In [ ]:
group_letters = ['A', 'B', 'C', 'D', 'E', 'F']
qualified = simulate_group_stage()

for code in sorted(qualified.keys()):
    group = group_letters[int(code[0])]
    rank  = int(code[1]) + 1
    
    print('{0:1s}{1:1d} {2:s}'.format(group, rank, qualified[code]))

In [ ]:
# If necessary for testing
actual_group_results = {
    '00': 'France',
    '01': 'Switzerland',
    '10': 'Wales',
    '11': 'England',
    '12': 'Slovakia',
    '20': 'Germany',
    '21': 'Poland',
    '22': 'Northern Ireland',
    '30': 'Croatia',
    '31': 'Spain',
    '40': 'Italy',
    '41': 'Belgium',
    '42': 'Republic of Ireland',
    '50': 'Hungary',
    '51': 'Iceland',
    '52': 'Portugal',
}

### Simulating the knockout stage

In [ ]:
from random import random

In [ ]:
A,B,C,D,E,F = 0,1,2,3,4,5

def third_placed(group_results):
    '''Return the comma-separated indices of the group, from which the 3rd-placed teams did qualify'''
    return ','.join(sorted([ k[0] for k, _ in group_results.items() if k[1] == '2' ]))

group_combinations = {
'0,1,2,3': [C,D,A,B],
'0,1,2,4': [C,A,B,E],
'0,1,2,5': [C,A,B,F],
'0,1,3,4': [D,A,B,E],
'0,1,3,5': [D,A,B,F],
'0,1,4,5': [E,A,B,F],
'0,2,3,4': [C,D,A,E],
'0,2,3,5': [C,D,A,F],
'0,2,4,5': [C,A,F,E],
'0,3,4,5': [D,A,F,E],
'1,2,3,4': [C,D,B,E],
'1,2,3,5': [C,D,B,F],
'1,2,4,5': [E,C,B,F],
'1,3,4,5': [E,D,B,F],
'2,3,4,5': [C,D,F,E]
}

def choose_3rd(opponent_group, all_third_placed):
    '''See the 3rd link to the Wikipedia above'''
    return str(group_combinations[all_third_placed][opponent_group]) + '2'
    
def create_bracket(group_results):
    gr = group_results
    tp = third_placed(group_results)
    
    return [(gr['01'], gr['21']),
            (gr['30'], gr[choose_3rd(opponent_group=3, all_third_placed=tp)]),
            (gr['10'], gr[choose_3rd(opponent_group=1, all_third_placed=tp)]),
            (gr['50'], gr['41']),
            (gr['20'], gr[choose_3rd(opponent_group=2, all_third_placed=tp)]),
            (gr['40'], gr['31']),
            (gr['00'], gr[choose_3rd(opponent_group=0, all_third_placed=tp)]),
            (gr['11'], gr['51'])]

def elo_probability(elo1, elo2):
    return expected_outcome(elo1, elo2)

def simulate_winner(teams, elo):
    team1 = teams[0]
    team2 = teams[1]
    elo1 = elo.get(team1,0)
    elo2 = elo.get(team2,0)
    
    probability_1_wins = elo_probability(elo1, elo2)
    winner = team1 if random() < probability_1_wins else team2
    return winner

def simulate_bracket(bracket, elo):
    new_bracket = bracket
    while True:
        winners = [simulate_winner(match, elo) for match in new_bracket]
        if len(winners) == 1:
            return winners[0]
        else:
            new_bracket = [(winners[2*i], winners[2*i+1]) for i in range(0, len(winners) // 2)]

def simulate_knockout(group_results, elo):
    '''Simulates the knockout phase and returns the winner of the tournament'''
    bracket = create_bracket(group_results)    
    winner = simulate_bracket(bracket, elo)
    return winner

### Simulator

In [ ]:
def simulate_tournament():
    knockout_bracket = simulate_group_stage()
    winner = simulate_knockout(knockout_bracket, elos)
    return winner

## Computing the title probabilities

In [ ]:
winners = []
max_iterations=1000

N = 0
while N < max_iterations:
    winners.append(simulate_tournament())
    N = N+1
    
    if (N<100 and N%10==0) or (N<1000 and N%100==0) or (N>=1000 and N%1000==0):
        teams, counts = np.unique(winners, return_counts=True)
        frequencies = counts / N
    
        team_ranking=sorted(range(0, np.size(teams)),key=lambda t:frequencies[t],reverse=True)
        for t in team_ranking:
            print('{2:4d};{0:17s};{1:.4f}'.format(teams[t], frequencies[t], N), flush=True)